# Thingamabob (model fuser)

by [aicrumb](https://twitter.com/aicrumb)

Fuse two Stable Diffusion models into one, ready to use with Doohickey! (or any other diffusers-based Stable Diffusion interface)

In [1]:
#@title Installation / Log in to 🤗
import os
from IPython.display import clear_output
import time
if not os.path.exists("installed.txt"):
    !pip install transformers diffusers -q
    !sudo apt-get install git-lfs
    !mkdir /content/output
    print("Installed libraries")
    time.sleep(1)
    clear_output(wait=False)

from huggingface_hub import notebook_login
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
#@title Load Models
# model fuser
from huggingface_hub import hf_hub_download
from diffusers import UNet2DConditionModel
import os 
import torch

if not os.path.exists("convert.py"):
    !pip install OmegaConf -q
    !curl https://raw.githubusercontent.com/huggingface/diffusers/main/scripts/convert_original_stable_diffusion_to_diffusers.py > convert.py

#@markdown put your model names here with a comma separating them (either `user/id/filename.ckpt` for CompVis style, or `user/id` for Diffusers style)
model_names_ = "CompVis/stable-diffusion-v1-4, hakurei/waifu-diffusion" #@param {"type":"string"}
model_names = [i.strip() for i in model_names_.split(",")]
unets = []
for i in model_names:
    print(i)
    if ".ckpt" in i:
        repo_id = "/".join(i.split("/")[:2])
        filename = "/".join(i.split("/")[2:])
        print("downloading model...")
        compvis_path = hf_hub_download(repo_id=repo_id, filename=filename)
        print("creating diffusers-style model... (this will take a while)")
        !python convert.py --checkpoint_path "$compvis_path" --dump_path "temp-model"
        !rm -r "$compvis_path"
        i = "temp-model/unet"
    if i=="temp-model/unet":
        unets.append(UNet2DConditionModel.from_pretrained(i).half())
    else:
        unets.append(UNet2DConditionModel.from_pretrained(i, subfolder="unet", use_auth_token=True))
clear_output(wait=False)

CompVis/stable-diffusion-v1-4


Downloading:   0%|          | 0.00/743 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

hakurei/waifu-diffusion


Downloading:   0%|          | 0.00/743 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

In [ ]:
#@title Mix! 🍲
theta_0 = unets[0].state_dict()
theta_1 = unets[1].state_dict()
alpha = 0.4 #@param

for key in theta_0.keys():
    if 'model' in key and key in theta_1:
        theta_0[key] = (1-alpha) * theta_0[key] + alpha * theta_1[key]

for key in theta_1.keys():
    if 'model' in key and key not in theta_0:
        theta_0[key] = theta_1[key]

del theta_1 
del unets[1]

# just in case idk
import gc
gc.collect()

unets[0].load_state_dict(theta_0)
unets[0].save_pretrained("/content/temp-model/unet")

In [ ]:
#@title Push to 🤗 hub.
#@markdown make the repository on https://hf.co before running this cell

from huggingface_hub import HfApi
api = HfApi()

repo = "user/my_model" #@param {"type":"string"}

readme = f"""
---
language:
- en
tags:
- stable-diffusion
- text-to-image
- thingamabob
license: bigscience-bloom-rail-1.0
inference: false

---

# {repo}
---
This is a fusion of {model_names_} made with [Doohickey](https://huggingface.co/doohickey/), alpha {alpha}.
"""
open("/content/temp-model/README.md", "w").write(readme)

api.upload_folder(
    folder_path="/content/temp-model",
    path_in_repo="",
    repo_id=repo,
    repo_type="model",
    ignore_patterns="**/logs/*.txt",
)

if this was used inside the doohickey notebook, before running other cells, you may want to restart the runtime / possibly factory restart to get rid of un-used files.